<a href="https://colab.research.google.com/github/DBagrecha/Instaminutes/blob/main/Insight_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Dropout

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


Reading the file for traing.

In [18]:
path='/content/drive/MyDrive/insights/train.csv'
df=pd.read_csv(path,sep = '|')
df=df.rename(columns={'Sentence ':'Sentence',' Category' :'Category'})
df.head()

,Sentence,Category
0,﻿The Project Gutenberg eBook of Pride and Prej...,1
1,This eBook is for the use of anyone anywhere i...,1
2,most other parts of the world at no cost and w...,0
3,"whatsoever. You may copy it, give it away or r...",0
4,of the Project Gutenberg License included with...,1


In [106]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [107]:
from nltk.tokenize import sent_tokenize

In [108]:
path = '/content/drive/MyDrive/insights/test.txt'
doc = open(path,'r').read()
data={'Sentence':'Category'}
sent=[]
for i in doc.split('\n'):
  for j in sent_tokenize(i):
    if(len(j.split())>3):
      sent.append(j)

In [110]:
a=list(df['Sentence'])
for i in sent:
  a.append(i)

11516

Converting sentences to array of Integers

In [111]:
tokens={}
c=0
w=[]
arr=[]
for i in a:
  w=[]
  for j in i.split():
    if (j in tokens.keys()):
      w.append(tokens[j])
      continue
    else:
      tokens[j]=c
      c+=1
      w.append(tokens[j])
  arr.append(w)

In [112]:
df['token_arr']=arr[:11393]

In [113]:
df.head()

,Sentence,Category,token_arr
0,﻿The Project Gutenberg eBook of Pride and Prej...,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
1,This eBook is for the use of anyone anywhere i...,1,"[11, 3, 12, 13, 14, 15, 4, 16, 17, 18, 14, 19,..."
2,most other parts of the world at no cost and w...,0,"[21, 22, 23, 4, 14, 24, 25, 26, 27, 6, 28, 29,..."
3,"whatsoever. You may copy it, give it away or r...",0,"[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 37, 4..."
4,of the Project Gutenberg License included with...,1,"[4, 14, 1, 2, 43, 44, 28, 45, 3, 39, 46, 25]"


In [114]:
X=sequence.pad_sequences(df['token_arr'],maxlen=40)
print('X shape:', X.shape)

X shape: (11393, 40)


MODEL

In [125]:
model=Sequential()
model.add(Embedding(len(tokens), 100))
model.add(GRU(50,recurrent_activation = 'sigmoid',dropout=0.2,input_shape=X.shape[1:]))
#model.add(GRU(50,recurrent_activation = 'relu',dropout=0.2))
#model.add(GRU(50,recurrent_activation = 'relu',dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [126]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 100)         1359900   
                                                                 
 gru_24 (GRU)                (None, 50)                22800     
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,382,751
Trainable params: 1,382,751
Non-trainable params: 0
_________________________________________________________________


In [127]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [128]:
model.fit(X,df['Category'],batch_size=100,epochs=10)

Epoch 1/10
114/114 [==============================] - 8s 53ms/step - loss: 0.5073 - accuracy: 0.7293
Epoch 2/10
114/114 [==============================] - 6s 53ms/step - loss: 0.1456 - accuracy: 0.9473
Epoch 3/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0547 - accuracy: 0.9829
Epoch 4/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0249 - accuracy: 0.9932
Epoch 5/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0177 - accuracy: 0.9943
Epoch 6/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0080 - accuracy: 0.9978
Epoch 7/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0041 - accuracy: 0.9992
Epoch 8/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0023 - accuracy: 0.9994
Epoch 9/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0028 - accuracy: 0.9993
Epoch 10/10
114/114 [==============================] - 6s 53ms/step - loss: 0.0040 - accura

In [129]:
Y=sequence.pad_sequences(arr[11393:],25)
Y.shape

(123, 25)

Getting insights from Input

In [130]:
p=model.predict(Y)

In [131]:
insights=[]
for i in range(len(p)):
  if (p[i]>0.5):
    insights.append(sent[i])